In [2]:
import numpy as np
import pandas as pd
import scipy as sc
import os
from datetime import date
from os import listdir
from os.path import isfile, join
import math
from scipy.optimize import curve_fit

In [3]:
def wistar_w(g, nu):
    if g == str("male"):
        x = nu-17
        y = [491.7, 438.3, 486.1, 572, 475.2, 468.6, 446, 431.7, 452.9, 413,
      417.7, 491.1, 495.9, 500.3, 548.2, 524.7]

    elif g == str("female"):
        x = nu-1
        y = [282.7, 340.6, 231.3, 301.9, 325.6, 328.7, 297.8, 295.2,
      306.7, 287.2, 276, 293.6, 301.9, 281.7]
    else:
        print("You fucked up the gender on weight optimizer.")
    return y[x]

In [ ]:
def d_day_index(f, n):
    for i1 in range(n):
        f.loc[i1, ['day_index']] = i1+1
    return f


def state_hour(f, n):
    for i2 in range(n + 1):
        dn = 24 * i2
        f.loc[dn:dn + 6, ['state']] = str('dark')
        f.loc[dn + 7:dn + 18, ['state']] = str('light')
        f.loc[dn + 19:dn + 24, ['state']] = str('dark')
    return f


def m_day_and_hour_index(f, n):
    h_counter = 1
    for i3 in range(n + 1):
        dn = 24 * i3
        f.loc[dn:dn + 23, ['day_index']] = i3+1
        for j3 in range(24):
            interval_m = dn + j3
            f.loc[interval_m:interval_m + 1, ['hour_index']] = h_counter
            h_counter = h_counter + 1
    return f


def state_(f, n):
    for i4 in range(n + 1):
        dn = 1440 * i4
        f.loc[dn:dn + 419, ['state']] = str('dark')
        f.loc[dn + 420:dn + 1139, ['state']] = str('light')
        f.loc[dn + 1140:dn + 1439, ['state']] = str('dark')
    return f


def day_and_hour_index(f, n):
    h_counter = 1
    for i5 in range(n + 1):
        dn = 1440 * i5
        f.loc[dn:dn + 1439, ['day_index']] = i5+1
        for j5 in range(24):
            interval_m = dn + (j5*60)
            f.loc[interval_m:interval_m + 60, ['hour_index']] = h_counter
            h_counter = h_counter + 1
    return f

def threshup_():
    #DOI 10.1211/jpp.60.1.0008
    th_u_comft = 3.38 + 0.52
    #th_u_stretched = 4.63 + 0.44
    #th_u_burst = 6.63 + 0.92
    return th_u_comft


def real_consump(val, w, p):
    a = np.nan
    if (p == 5):
        a = (val*0.05*0.789)/(w) *1000
    elif (p == 10):
        a = (val*0.1*0.789)/(w) *1000
    elif ( p == 20 ):
        a = (val*0.2*0.789)/(w) *1000
    else:
        print("You messed up the input in real_consump function")
    return(a)



def writer_(main_df, list_of_them, number, method, a_n, path):
    print(number)
    file = pd.read_excel("{}\{}".format(path, list_of_them[number]), skiprows=35)
    if method == str('dep_onlywater'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i6 in range(len(extracted_a1)):
            ind0 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i6]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i6]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind0, 'water'] = extracted_a1['water'][list_index[i6]]
    elif method == str('ade_only'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5%',
                                     '[ml].2': 'alcohol_10%', '[ml].3': 'alcohol_20%'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i7 in range(len(extracted_a1)):
            ind1 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i7]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i7]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind1, 'water'] = extracted_a1['water'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_5%'] = extracted_a1['alcohol_5%'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_10%'] = extracted_a1['alcohol_10%'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_20%'] = extracted_a1['alcohol_20%'][list_index[i7]]
    elif method == str('ade_oxy'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5%',
                                     '[ml].2': 'alcohol_10%', '[ml].3': 'alcohol_20%'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i8 in range(len(extracted_a1)):
            ind2 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i8]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i8]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind2, 'water'] = extracted_a1['water'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_5%'] = extracted_a1['alcohol_5%'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_10%'] = extracted_a1['alcohol_10%'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_20%'] = extracted_a1['alcohol_20%'][list_index[i8]]
            main_df.loc[ind2, 'oxytocin'] = str('applied')
    elif method == str('dep_quinine'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i9 in range(len(extracted_a1)):
            ind3 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i9]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i9]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind3, 'water'] = extracted_a1['water'][list_index[i9]]
            main_df.loc[ind3, 'quinine'] = str('applied')
    elif method == str('ade_quinine'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5%',
                                     '[ml].2': 'alcohol_10%', '[ml].3': 'alcohol_20%'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i10 in range(len(extracted_a1)):
            ind4 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i10]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i10]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind4, 'water'] = extracted_a1['water'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_5%'] = extracted_a1['alcohol_5%'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_10%'] = extracted_a1['alcohol_10%'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_20%'] = extracted_a1['alcohol_20%'][list_index[i10]]
            main_df.loc[ind4, 'quinine'] = str('applied')
    else:
        print("{} is non of them".format(list_of_them[number]))
    return main_df

In [ ]:
date_i = date(2020, 2, 12)
date_f = date(2020, 11, 17)

delta = date_f - date_i
number_of_days = int(delta.days)+1
print("Experiement was {} days long".format(number_of_days))

total_length = number_of_days * 1440      #length of dataframe for minute time_scale
df = pd.DataFrame(index = range(total_length), columns = ['date', 'time', 'day_index', 'hour_index',
                                                         'animal', 'box', 'strain',
                                                         'state', 'oxytocin', 'quinine', 'water',
                                                         'alcohol_5%', 'alcohol_10%', 'alcohol_20%',
                                                         'locomotive'])

f = state_(df, number_of_days)
df = day_and_hour_index(df, number_of_days)
df['animal'] = animal
df['box'] = box
df['strain'] = strain
df['date'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%Y-%m-%d')
df['time'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%H:%M:%S')

path_ = os.getcwd()
path_ = path_+'\\data\\2021'
onlyfiles = [f for f in listdir("{}".format(path_))
             if isfile(join("{}".format(path_), f))]

In [ ]:
df = pd.read_excel("{}\{}".format(path_, onlyfiles[26]), skiprows=35)

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5',
                                     '[ml].2': 'alcohol_10', '[ml].3': 'alcohol_20'})
for i in range(0,len(df)-1,1):
    df.loc[i, 'water'] = df.loc[i+1, 'water'] - df.loc[i, 'water']
    df.loc[i, 'alcohol_5%'] = df.loc[i+1, 'alcohol_5%'] - df.loc[i, 'alcohol_5%']
    df.loc[i, 'alcohol_10%'] = df.loc[i+1, 'alcohol_10%'] - df.loc[i, 'alcohol_10%']
    df.loc[i, 'alcohol_20%'] = df.loc[i+1, 'alcohol_20%'] - df.loc[i, 'alcohol_20%']
    
df.to_excel(r'{}\dataframes\drinkometer_df_raw.xlsx'.format(path_), index=False, header=True)


In [ ]:
df = writer_(df, onlyfiles, 2, str('ade_only'), box, path_)

df1 = pd.DataFrame(index=range(total_length), columns=['date', 'time', 'day_index', 'hour_index', 'animal', 'box',
                                                       'strain', 'state', 'oxytocin', 'quinine', 'water',
                                                       'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])


In [ ]:
df1 = state_(df1, number_of_days)
df1 = day_and_hour_index(df1, number_of_days)
df1['animal'] = animal
df1['box'] = box
df1['strain'] = strain
df1['date'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%Y-%m-%d')
df1['time'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%H:%M:%S')
df1['quinine'] = df['quinine']
df1['oxytocin'] = df['oxytocin']
df1['state'] = df['state']

for hour in range(1, number_of_days * 24 + 1):
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['water'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'water'] - df.loc[list_[i], 'water']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'water'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_5%'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_5%'] - df.loc[list_[i], 'alcohol_5%']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_5%'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_10%'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_10%'] - df.loc[list_[i], 'alcohol_10%']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_10%'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_20%'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_20%'] - df.loc[list_[i], 'alcohol_20%']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_20%'] = temp_val

In [ ]:
threshold_low = 0.08
threshold_up = threshup_()
total_length_h = number_of_days * 24

df_h = pd.DataFrame(index=range(number_of_days * 24), columns=['date', 'time', 'day_index', 'hour_index', 'animal',
                                                               'box', 'strain', 'state', 'oxytocin', 'quinine',
                                                               'water', 'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])
df_h = m_day_and_hour_index(df_h, number_of_days)
df_h = state_hour(df_h, number_of_days)

df_h['animal'] = animal
df_h['box'] = box
df_h['strain'] = strain
df_h['date'] = pd.date_range("2020-02-12", periods=number_of_days * 24, freq="H").strftime('%Y-%m-%d')
df_h['time'] = pd.date_range("2020-02-12", periods=number_of_days * 24, freq="H").strftime('%H:%M:%S')

hours = sorted(set(np.array(df1.loc[(df1['quinine'] == str('applied'))].hour_index)))
for i in range(len(hours)):
    df_h.loc[hours[i]-1, 'quinine'] = str('applied')

hours = sorted(set(np.array(df1.loc[(df1['oxytocin'] == str('applied'))].hour_index)))
for i in range(len(hours)):
    df_h.loc[hours[i]-1, 'oxytocin'] = str('applied')


for hour in range(1, number_of_days*24+1):
    w_w = wistar_w(gender, animal)
    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            df_h.loc[ind, 'water'] = _temp
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_5%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5%'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up ):
            temp__ = real_consump(_temp, w_w, 5)
            df_h.loc[ind, 'alcohol_5%'] = temp__
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_10%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10%'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            temp__ = real_consump(_temp, w_w, 10)
            df_h.loc[ind, 'alcohol_10%'] = temp__
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_20%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20%'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            temp__ = real_consump(_temp, w_w, 20)
            df_h.loc[ind, 'alcohol_20%'] = temp__
            _temp = np.nan


In [ ]:
df_d_light = pd.DataFrame(index=range(number_of_days), columns=['date', 'day_index', 'animal', 'box',
                                                                'strain', 'state', 'oxytocin', 'quinine', 'water',
                                                                'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])

df_d_dark = pd.DataFrame(index=range(number_of_days), columns=['date', 'day_index', 'animal', 'box',
                                                               'strain', 'state', 'oxytocin', 'quinine',
                                                               'water', 'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])

df_d_light = d_day_index(df_d_light, number_of_days)
df_d_dark = d_day_index(df_d_dark, number_of_days)

df_d_light['animal'] = animal
df_d_light['box'] = box
df_d_light['strain'] = strain
df_d_light['state'] = str('light')
df_d_light['date'] = pd.date_range("2020-02-12", periods=number_of_days, freq="D").strftime('%Y-%m-%d')

df_d_dark['animal'] = animal
df_d_dark['box'] = box
df_d_dark['strain'] = strain
df_d_dark['state'] = str('dark')
df_d_dark['date'] = pd.date_range("2020-02-12", periods=number_of_days, freq="D").strftime('%Y-%m-%d')


days = sorted(set(np.array(df1.loc[(df1['quinine'] == str('applied'))].day_index)))
for i in range(len(days)):
    df_d_dark.loc[days[i]-1, 'quinine'] = str('applied')
    df_d_light.loc[days[i]-1, 'quinine'] = str('applied')

days = sorted(set(np.array(df1.loc[(df1['oxytocin'] == str('applied'))].day_index)))
for i in range(len(days)):
    df_d_dark.loc[days[i]-1, 'oxytocin'] = str('applied')
    df_d_light.loc[days[i]-1, 'oxytocin'] = str('applied')


for day in range(1, number_of_days+1):
    list_ = []
    ind = np.array(df_d_light.loc[(df_d_light['day_index'] == day)].index)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'water'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'water'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_5%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5%'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_5%'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_5%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5%'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_5%'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_10%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10%'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_10%'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_10%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10%'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_10%'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_20%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20%'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_20%'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_20%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20%'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_20%'] = np.nansum(temp_arr)

for day in range(1, number_of_days+1):
    list_ = []
    ind = np.array(df_d_light.loc[(df_d_light['day_index'] == day)].index)
    df_extracted = df1_wl.loc[(df1_wl['day_index'] == day) & (df1_wl['state'] == str('light')) & (df1_wl['locomotive'].notnull())]
    temp_arr = np.array(df_extracted['locomotive'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'locomotive'] = np.nansum(temp_arr)
    df_extracted = df1_wl.loc[(df1_wl['day_index'] == day) & (df1_wl['state'] == str('dark')) & (df1_wl['locomotive'].notnull())]
    temp_arr = np.array(df_extracted['locomotive'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'locomotive'] = np.nansum(temp_arr)

In [ ]:
df1.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_consumptiontt.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_h.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_hourly.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_d_dark.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_daily_dark.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_d_light.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_daily_light.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)

In [ ]:
df1_d = pd.read_excel("results/drinkometer_df_w_1001_1_daily_dark_wloco.xlsx")
df1_l = pd.read_excel("results/drinkometer_df_w_1001_1_daily_light_wloco.xlsx")

df2_d = pd.read_excel("results/drinkometer_df_w_1002_2_daily_dark_wloco.xlsx")
df2_l = pd.read_excel("results/drinkometer_df_w_1002_2_daily_light_wloco.xlsx")

df3_d = pd.read_excel("results/drinkometer_df_w_1003_3_daily_dark_wloco.xlsx")
df3_l = pd.read_excel("results/drinkometer_df_w_1003_3_daily_light_wloco.xlsx")

df4_d = pd.read_excel("results/drinkometer_df_w_1004_4_daily_dark_wloco.xlsx")
df4_l = pd.read_excel("results/drinkometer_df_w_1004_4_daily_light_wloco.xlsx")

df5_d = pd.read_excel("results/drinkometer_df_w_1005_5_daily_dark_wloco.xlsx")
df5_l = pd.read_excel("results/drinkometer_df_w_1005_5_daily_light_wloco.xlsx")

df6_d = pd.read_excel("results/drinkometer_df_w_1006_6_daily_dark_wloco.xlsx")
df6_l = pd.read_excel("results/drinkometer_df_w_1006_6_daily_light_wloco.xlsx")

df8_d = pd.read_excel("results/drinkometer_df_w_1008_8_daily_dark_wloco.xlsx")
df8_l = pd.read_excel("results/drinkometer_df_w_1008_8_daily_light_wloco.xlsx")

df9_d = pd.read_excel("results/drinkometer_df_w_1009_9_daily_dark_wloco.xlsx")
df9_l = pd.read_excel("results/drinkometer_df_w_1009_9_daily_light_wloco.xlsx")

df10_d = pd.read_excel("results/drinkometer_df_w_1010_10_daily_dark_wloco.xlsx")
df10_l = pd.read_excel("results/drinkometer_df_w_1010_10_daily_light_wloco.xlsx")

df11_d = pd.read_excel("results/drinkometer_df_w_1011_11_daily_dark_wloco.xlsx")
df11_l = pd.read_excel("results/drinkometer_df_w_1011_11_daily_light_wloco.xlsx")

df12_d = pd.read_excel("results/drinkometer_df_w_1012_12_daily_dark_wloco.xlsx")
df12_l = pd.read_excel("results/drinkometer_df_w_1012_12_daily_light_wloco.xlsx")

df13_d = pd.read_excel("results/drinkometer_df_w_1013_13_daily_dark_wloco.xlsx")
df13_l = pd.read_excel("results/drinkometer_df_w_1013_13_daily_light_wloco.xlsx")

df14_d = pd.read_excel("results/drinkometer_df_w_1014_14_daily_dark_wloco.xlsx")
df14_l = pd.read_excel("results/drinkometer_df_w_1014_14_daily_light_wloco.xlsx")

df15_d = pd.read_excel("results/drinkometer_df_w_1015_15_daily_dark_wloco.xlsx")
df15_l = pd.read_excel("results/drinkometer_df_w_1015_15_daily_light_wloco.xlsx")

df17_d = pd.read_excel("results/drinkometer_df_w_1017_17_daily_dark_wloco.xlsx")
df17_l = pd.read_excel("results/drinkometer_df_w_1017_17_daily_light_wloco.xlsx")

df18_d = pd.read_excel("results/drinkometer_df_w_1018_18_daily_dark_wloco.xlsx")
df18_l = pd.read_excel("results/drinkometer_df_w_1018_18_daily_light_wloco.xlsx")

df19_d = pd.read_excel("results/drinkometer_df_w_1019_19_daily_dark_wloco.xlsx")
df19_l = pd.read_excel("results/drinkometer_df_w_1019_19_daily_light_wloco.xlsx")

df20_d = pd.read_excel("results/drinkometer_df_w_1020_20_daily_dark_wloco.xlsx")
df20_l = pd.read_excel("results/drinkometer_df_w_1020_20_daily_light_wloco.xlsx")

df21_d = pd.read_excel("results/drinkometer_df_w_1021_21_daily_dark_wloco.xlsx")
df21_l = pd.read_excel("results/drinkometer_df_w_1021_21_daily_light_wloco.xlsx")

df22_d = pd.read_excel("results/drinkometer_df_w_1022_22_daily_dark_wloco.xlsx")
df22_l = pd.read_excel("results/drinkometer_df_w_1022_22_daily_light_wloco.xlsx")

df23_d = pd.read_excel("results/drinkometer_df_w_1023_23_daily_dark_wloco.xlsx")
df23_l = pd.read_excel("results/drinkometer_df_w_1023_23_daily_light_wloco.xlsx")

df24_d = pd.read_excel("results/drinkometer_df_w_1024_24_daily_dark_wloco.xlsx")
df24_l = pd.read_excel("results/drinkometer_df_w_1024_24_daily_light_wloco.xlsx")

df25_d = pd.read_excel("results/drinkometer_df_w_1025_25_daily_dark_wloco.xlsx")
df25_l = pd.read_excel("results/drinkometer_df_w_1025_25_daily_light_wloco.xlsx")

df26_d = pd.read_excel("results/drinkometer_df_w_1026_26_daily_dark_wloco.xlsx")
df26_l = pd.read_excel("results/drinkometer_df_w_1026_26_daily_light_wloco.xlsx")

df27_d = pd.read_excel("results/drinkometer_df_w_1027_27_daily_dark_wloco.xlsx")
df27_l = pd.read_excel("results/drinkometer_df_w_1027_27_daily_light_wloco.xlsx")

df28_d = pd.read_excel("results/drinkometer_df_w_1028_28_daily_dark_wloco.xlsx")
df28_l = pd.read_excel("results/drinkometer_df_w_1028_28_daily_light_wloco.xlsx")

df29_d = pd.read_excel("results/drinkometer_df_w_1029_29_daily_dark_wloco.xlsx")
df29_l = pd.read_excel("results/drinkometer_df_w_1029_29_daily_light_wloco.xlsx")

df30_d = pd.read_excel("results/drinkometer_df_w_1030_30_daily_dark_wloco.xlsx")
df30_l = pd.read_excel("results/drinkometer_df_w_1030_30_daily_light_wloco.xlsx")

df31_d = pd.read_excel("results/drinkometer_df_w_1031_31_daily_dark_wloco.xlsx")
df31_l = pd.read_excel("results/drinkometer_df_w_1031_31_daily_light_wloco.xlsx")

df32_d = pd.read_excel("results/drinkometer_df_w_1032_32_daily_dark_wloco.xlsx")
df32_l = pd.read_excel("results/drinkometer_df_w_1032_32_daily_light_wloco.xlsx")



In [ ]:
df_m = pd.DataFrame(index=range(2*280), columns=['date', 'day_index', 'strain', 'state', 'oxytocin', 'quinine',
                                                'w_1', 'a5_1', 'a10_1', 'a20_1', 'loco_1',
                                                'w_2', 'a5_2', 'a10_2', 'a20_2', 'loco_2',
                                                'w_3', 'a5_3', 'a10_3', 'a20_3', 'loco_3',
                                                'w_4', 'a5_4', 'a10_4', 'a20_4', 'loco_4',
                                                'w_5', 'a5_5', 'a10_5', 'a20_5', 'loco_5',
                                                'w_6', 'a5_6', 'a10_6', 'a20_6', 'loco_6',
                                                'w_8', 'a5_8', 'a10_8', 'a20_8', 'loco_8',
                                                'w_9', 'a5_9', 'a10_9', 'a20_9', 'loco_9',
                                                'w_10', 'a5_10', 'a10_10', 'a20_10', 'loco_10',
                                                'w_11', 'a5_11', 'a10_11', 'a20_11', 'loco_11',
                                                'w_12', 'a5_12', 'a10_12', 'a20_12', 'loco_12',
                                                'w_13', 'a5_13', 'a10_13', 'a20_13', 'loco_13',
                                                'w_14', 'a5_14', 'a10_14', 'a20_14', 'loco_14',
                                                'w_15', 'a5_15', 'a10_15', 'a20_15', 'loco_15',
                                                'w_17', 'a5_17', 'a10_17', 'a20_17', 'loco_17',
                                                'w_18', 'a5_18', 'a10_18', 'a20_18', 'loco_18',
                                                'w_19', 'a5_19', 'a10_19', 'a20_19', 'loco_19',
                                                'w_20', 'a5_20', 'a10_20', 'a20_20', 'loco_20',
                                                'w_21', 'a5_21', 'a10_21', 'a20_21', 'loco_21',
                                                'w_22', 'a5_22', 'a10_22', 'a20_22', 'loco_22',
                                                'w_23', 'a5_23', 'a10_23', 'a20_23', 'loco_23',
                                                'w_24', 'a5_24', 'a10_24', 'a20_24', 'loco_24',
                                                'w_25', 'a5_25', 'a10_25', 'a20_25', 'loco_25',
                                                'w_26', 'a5_26', 'a10_26', 'a20_26', 'loco_26',
                                                'w_27', 'a5_27', 'a10_27', 'a20_27', 'loco_27',
                                                'w_28', 'a5_28', 'a10_28', 'a20_28', 'loco_28',
                                                'w_29', 'a5_29', 'a10_29', 'a20_29', 'loco_29',
                                                'w_30', 'a5_30', 'a10_30', 'a20_30', 'loco_30',
                                                'w_31', 'a5_31', 'a10_31', 'a20_31', 'loco_31',
                                                'w_32', 'a5_32', 'a10_32', 'a20_32', 'loco_32'])


In [ ]:
for i in range(280):
    df_m.loc[2*i, 'state'] = str('dark')
    df_m.loc[2*i+1, 'state'] = str('light')
    df_m.loc[2*i, 'date'] = df1_l.loc[i, 'date']
    df_m.loc[2*i+1, 'date'] = df1_l.loc[i, 'date']
    df_m.loc[2*i, 'day_index'] = df1_l.loc[i, 'day_index']
    df_m.loc[2*i+1, 'day_index'] = df1_l.loc[i, 'day_index']
    df_m.loc[2*i, 'strain'] = df1_l.loc[i, 'strain']
    df_m.loc[2*i+1, 'strain'] = df1_l.loc[i, 'strain']
    df_m.loc[2*i, 'oxytocin'] = df1_l.loc[i, 'oxytocin']
    df_m.loc[2*i+1, 'oxytocin'] = df1_l.loc[i, 'oxytocin']
    df_m.loc[2*i, 'quinine'] = df1_l.loc[i, 'quinine']
    df_m.loc[2*i+1, 'quinine'] = df1_l.loc[i, 'quinine']
    
    df_m.loc[2*i, 'w_1'] = df1_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_1'] = df1_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_1'] = df1_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_1'] = df1_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_1'] = df1_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_1'] = df1_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_1'] = df1_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_1'] = df1_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_1'] = df1_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_1'] = df1_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_2'] = df2_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_2'] = df2_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_2'] = df2_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_2'] = df2_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_2'] = df2_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_2'] = df2_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_2'] = df2_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_2'] = df2_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_2'] = df2_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_2'] = df2_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_3'] = df3_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_3'] = df3_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_3'] = df3_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_3'] = df3_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_3'] = df3_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_3'] = df3_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_3'] = df3_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_3'] = df3_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_3'] = df3_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_3'] = df3_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_4'] = df4_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_4'] = df4_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_4'] = df4_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_4'] = df4_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_4'] = df4_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_4'] = df4_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_4'] = df4_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_4'] = df4_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_4'] = df4_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_4'] = df4_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_5'] = df5_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_5'] = df5_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_5'] = df5_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_5'] = df5_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_5'] = df5_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_5'] = df5_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_5'] = df5_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_5'] = df5_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_5'] = df5_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_5'] = df5_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_6'] = df6_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_6'] = df6_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_6'] = df6_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_6'] = df6_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_6'] = df6_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_6'] = df6_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_6'] = df6_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_6'] = df6_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_6'] = df6_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_6'] = df6_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_8'] = df8_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_8'] = df8_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_8'] = df8_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_8'] = df8_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_8'] = df8_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_8'] = df8_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_8'] = df8_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_8'] = df8_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_8'] = df8_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_8'] = df8_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_9'] = df9_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_9'] = df9_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_9'] = df9_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_9'] = df9_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_9'] = df9_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_9'] = df9_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_9'] = df9_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_9'] = df9_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_9'] = df9_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_9'] = df9_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_10'] = df10_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_10'] = df10_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_10'] = df10_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_10'] = df10_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_10'] = df10_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_10'] = df10_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_10'] = df10_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_10'] = df10_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_10'] = df10_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_10'] = df10_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_11'] = df11_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_11'] = df11_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_11'] = df11_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_11'] = df11_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_11'] = df11_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_11'] = df11_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_11'] = df11_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_11'] = df11_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_11'] = df11_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_11'] = df11_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_12'] = df12_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_12'] = df12_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_12'] = df12_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_12'] = df12_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_12'] = df12_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_12'] = df12_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_12'] = df12_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_12'] = df12_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_12'] = df12_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_12'] = df12_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_13'] = df13_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_13'] = df13_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_13'] = df13_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_13'] = df13_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_13'] = df13_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_13'] = df13_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_13'] = df13_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_13'] = df13_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_13'] = df13_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_13'] = df13_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_14'] = df14_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_14'] = df14_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_14'] = df14_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_14'] = df14_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_14'] = df14_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_14'] = df14_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_14'] = df14_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_14'] = df14_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_14'] = df14_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_14'] = df14_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_15'] = df15_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_15'] = df15_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_15'] = df15_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_15'] = df15_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_15'] = df15_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_15'] = df15_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_15'] = df15_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_15'] = df15_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_15'] = df15_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_15'] = df15_l.loc[i, 'locomotive']

    df_m.loc[2*i, 'w_17'] = df17_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_17'] = df17_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_17'] = df17_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_17'] = df17_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_17'] = df17_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_17'] = df17_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_17'] = df17_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_17'] = df17_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_17'] = df17_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_17'] = df17_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_18'] = df18_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_18'] = df18_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_18'] = df18_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_18'] = df18_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_18'] = df18_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_18'] = df18_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_18'] = df18_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_18'] = df18_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_18'] = df18_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_18'] = df18_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_19'] = df19_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_19'] = df19_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_19'] = df19_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_19'] = df19_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_19'] = df19_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_19'] = df19_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_19'] = df19_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_19'] = df19_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_19'] = df19_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_19'] = df19_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_20'] = df20_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_20'] = df20_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_20'] = df20_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_20'] = df20_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_20'] = df20_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_20'] = df20_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_20'] = df20_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_20'] = df20_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_20'] = df20_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_20'] = df20_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_21'] = df21_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_21'] = df21_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_21'] = df21_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_21'] = df21_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_21'] = df21_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_21'] = df21_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_21'] = df21_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_21'] = df21_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_21'] = df21_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_21'] = df21_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_22'] = df22_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_22'] = df22_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_22'] = df22_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_22'] = df22_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_22'] = df22_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_22'] = df22_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_22'] = df22_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_22'] = df22_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_22'] = df22_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_22'] = df22_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_23'] = df23_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_23'] = df23_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_23'] = df23_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_23'] = df23_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_23'] = df23_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_23'] = df23_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_23'] = df23_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_23'] = df23_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_23'] = df23_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_23'] = df23_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_24'] = df24_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_24'] = df24_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_24'] = df24_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_24'] = df24_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_24'] = df24_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_24'] = df24_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_24'] = df24_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_24'] = df24_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_24'] = df24_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_24'] = df24_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_25'] = df25_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_25'] = df25_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_25'] = df25_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_25'] = df25_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_25'] = df25_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_25'] = df25_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_25'] = df25_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_25'] = df25_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_25'] = df25_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_25'] = df25_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_26'] = df26_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_26'] = df26_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_26'] = df26_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_26'] = df26_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_26'] = df26_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_26'] = df26_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_26'] = df26_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_26'] = df26_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_26'] = df26_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_26'] = df26_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_27'] = df27_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_27'] = df27_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_27'] = df27_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_27'] = df27_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_27'] = df27_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_27'] = df27_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_27'] = df27_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_27'] = df27_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_27'] = df27_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_27'] = df27_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_28'] = df28_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_28'] = df28_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_28'] = df28_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_28'] = df28_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_28'] = df28_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_28'] = df28_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_28'] = df28_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_28'] = df28_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_28'] = df28_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_28'] = df28_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_29'] = df29_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_29'] = df29_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_29'] = df29_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_29'] = df29_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_29'] = df29_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_29'] = df29_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_29'] = df29_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_29'] = df29_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_29'] = df29_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_29'] = df29_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_30'] = df30_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_30'] = df30_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_30'] = df30_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_30'] = df30_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_30'] = df30_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_30'] = df30_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_30'] = df30_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_30'] = df30_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_30'] = df30_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_30'] = df30_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_31'] = df31_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_31'] = df31_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_31'] = df31_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_31'] = df31_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_31'] = df31_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_31'] = df31_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_31'] = df31_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_31'] = df31_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_31'] = df31_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_31'] = df31_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_32'] = df32_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_32'] = df32_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_32'] = df32_d.loc[i, 'alcohol-5%']
    df_m.loc[2*i+1, 'a5_32'] = df32_l.loc[i, 'alcohol-5%']
    df_m.loc[2*i, 'a10_32'] = df32_d.loc[i, 'alcohol-10%']
    df_m.loc[2*i+1, 'a10_32'] = df32_l.loc[i, 'alcohol-10%']
    df_m.loc[2*i, 'a20_32'] = df32_d.loc[i, 'alcohol-20%']
    df_m.loc[2*i+1, 'a20_32'] = df32_l.loc[i, 'alcohol-20%']
    df_m.loc[2*i, 'loco_32'] = df32_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_32'] = df32_l.loc[i, 'locomotive']